In [1]:
import xarray as xr
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import xlrd
import pyisotopomer

import bokeh
import bokeh.io
import bokeh.plotting

In [2]:
# Setting up Bokeh to run inline

bokeh.io.output_notebook()

Loading BokehJS ...

# Inputs

## <span style="color:cyan">INPUT: Isodat files</span>
Paths to all Isodat output files you want to analyze with a common set of standards. Noah recommends using 5-day chunks.

In [4]:
input_list = ['raw_benguela_data/250218_V_Benguela24_Stn21.xlsx',
             'raw_benguela_data/250219_V_Benguela24_Stn21_20.xlsx',
             'raw_benguela_data/250220_V_Benguela24_Stn20_22.xlsx',
             'raw_benguela_data/250221_V_Benguela24_St20.xlsx',
             'raw_benguela_data/250224_V_Benguela24_St22.xls',
             'raw_benguela_data/250225_V_Benguela24_St22.xls']

## <span style="color:cyan">INPUT: Output file names</span>
Set where you want pyisotopomer and final processed output to be saved. Keeping `None` for the pyisotopomer output locations will save with the default name in your working directory.

In [3]:
chunkname= "250218_250225_Benguela24"

scrambling_output_file="pyisotopomer_output/scrambling_" + chunkname + ".xlsx"
isotopomers_output_file="pyisotopomer_output/isotopomers_" + chunkname + ".xlsx"
processed_output_file="processed_benguela_data/" + chunkname + ".csv"

## <span style="color:cyan">INPUT: Size series and scrambling </span>
Path to that month's scrambling and size correction output. These are found in the Google Drive in `Project Folders/A01 standards and troubleshooting/Scrambling runs` and `Project Folders/A01 standards and troubleshooting/Size series & sensitivity`.

In [5]:
sizeseries_excel = 'scrambling_sizecorr/250211 N2O Size Correction and Sensitivity.xlsx'
scrambling_excel = 'scrambling_sizecorr/250211 A01 Scrambling.xlsx'

Make sure these values match what's in the spreadsheet. If the cells in the template got shifted, you'll have to adjust the next couple cells.

In [6]:
sizeseries_df = pd.read_excel(sizeseries_excel, usecols="L:R", skiprows=46, nrows=2)
sizeseries_df

,Unnamed: 11,45R,Unnamed: 13,46R,Unnamed: 15,31R,Unnamed: 17
0,SLOPE,0.000038,0.999891,0.000040,1.001197,0.000020,0.997351
1,SLOPE ERROR,0.000010,0.000141,0.000007,0.000090,0.000024,0.000325


In [7]:
# getting size series slope values

sizecorr_slope45R = sizeseries_df["45R"][0]
sizecorr_slope46R = sizeseries_df["46R"][0]
sizecorr_slope31R = sizeseries_df["31R"][0]

print("45R slope: " + str(sizecorr_slope45R))
print("46R slope: " + str(sizecorr_slope46R))
print("31R slope: " + str(sizecorr_slope31R))


45R slope: 3.773561488e-05
46R slope: 4.032722163e-05
31R slope: 1.971733761e-05


In [8]:
scrambling_df = pd.read_excel(scrambling_excel, usecols="AL:AQ", skiprows=1)
scrambling_df[0:3] #gamma and kappa in all rows should be the same, but feel free to check more

,size corrected 31R,size corrected 45R,size corrected 46R,D17O,gamma,kappa
0,0.003729,0.007742,0.002097,0,0.1799,0.087173
1,0.003729,0.007743,0.002097,0,0.1799,0.087173
2,0.003728,0.007745,0.002098,0,0.1799,0.087173


In [9]:
gamma_initial_guess = scrambling_df["gamma"][0]
kappa_initial_guess = scrambling_df["kappa"][0]

print("gamma initial guess: " + str(gamma_initial_guess))
print("kappa initial guess: " + str(kappa_initial_guess))

gamma initial guess: 0.179899673101926
kappa initial guess: 0.08717302279090275


## Concentration constants

In [51]:
conc_slopes_df = pd.read_excel(sizeseries_excel, usecols="F:I", skiprows=38, nrows=5)
conc_slopes_df

,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8
0,slope,0.413459,0.795369,int
1,slope_se,0.017374,0.233450,int_se
2,r^2,0.982648,0.495724,y_se
3,F statistic,566.304379,10.000000,df
4,ss_reg,139.164948,2.457423,ss_resid


In [11]:
conc_slope = conc_slopes_df.iloc[0,1]
conc_slope_err = conc_slopes_df.iloc[1,1]

conc_int = conc_slopes_df.iloc[0,2]
conc_int_err = conc_slopes_df.iloc[1,2]

print("slope (nmol/Vs): " + str(conc_slope))
print("slope error (nmol/Vs): " + str(conc_slope_err))
print("intercept (nmol): " + str(conc_int))
print("intercept error (nmol): " + str(conc_int_err))

slope (nmol/Vs): 0.4134590069
slope error (nmol/Vs): 0.01737430724048442
intercept (nmol): 0.7953691823131895
intercept error (nmol): 0.23344961388830332


## Reference N$_2$O tank values
These stay constant unless the reference tank gets changed, which won't happen for a while.

In [12]:
N2O_ref31R = 0.00373376282567055
N2O_ref45R = 0.00774102494962263
N2O_ref46R = 0.00210129522157562

## Reference values for standard gases

Currently includes S2, AESW, and 1000 ppm. Add others if you need them.

In [13]:
### Constants
# Consider running this elsewhere and pulling these values in from a CSV

# Known values for standards

std_const = pd.DataFrame(index=["S2", "AESW", "1000ppm"], columns=["ref_tag", "d15Na", "d15Nb", "d18O"])
std_const["ref_tag"] = std_const.index

std_const.at['S2', 'd15Na'], std_const.at['S2', 'd15Nb'], std_const.at['S2', 'd18O'] = 5.55, -12.87, 32.73
std_const.at['AESW', 'd15Na'], std_const.at['AESW', 'd15Nb'], std_const.at['AESW', 'd18O'] = 15.6, -2.3, 44.4
std_const.at['1000ppm', 'd15Na'], std_const.at['1000ppm', 'd15Nb'], std_const.at['1000ppm', 'd18O'] = -0.11, -0.45, 37.83

# Derived values

std_const['d15Nbulk'] = std_const[['d15Na', 'd15Nb']].mean(axis=1)
std_const['SP'] = std_const['d15Na'] - std_const['d15Nb']

std_const['15Ralpha'] = (std_const['d15Na']/1000 + 1)*0.0036765
std_const['15Rbeta'] = (std_const['d15Nb']/1000 + 1)*0.0036765
std_const['18R'] = (std_const['d18O']/1000 + 1)*0.0020052
std_const['17R'] = (std_const['d18O']/1000 + 1)**0.516 * 0.0003799
std_const['45R'] = std_const['15Ralpha'] + std_const['15Rbeta'] + std_const['17R']
std_const['46R'] = ((std_const['15Ralpha'] + std_const['15Rbeta'])*std_const['17R'] 
                    + std_const['18R'] + std_const['15Ralpha']*std_const['15Rbeta'])

std_const['45R/45R'] = std_const['45R']/N2O_ref45R
std_const['46R/46R'] = std_const['46R']/N2O_ref46R

std_const

,ref_tag,d15Na,d15Nb,d18O,d15Nbulk,SP,15Ralpha,15Rbeta,18R,17R,45R,46R,45R/45R,46R/46R
S2,S2,5.55,-12.87,32.73,-3.66,18.42,0.003697,0.003629,0.002071,0.000386,0.007712,0.002087,0.996296,0.993233
AESW,AESW,15.6,-2.3,44.4,6.65,17.9,0.003734,0.003668,0.002094,0.000389,0.00779,0.002111,1.00638,1.004525
1000ppm,1000ppm,-0.11,-0.45,37.83,-0.28,0.34,0.003676,0.003675,0.002081,0.000387,0.007738,0.002097,0.999634,0.998152


# Preprocessing
## Reading in Isodat data

Read the data into a Pandas dataframe and concatenate.

In [14]:
df_sample = pd.DataFrame()
df_std = pd.DataFrame()

for filename in input_list:
    single_df_sample = pd.read_excel(filename, sheet_name="160hdspcN2OrR314546sample_V3_15")
    single_df_std = pd.read_excel(filename, sheet_name="160hdspcN2OrR314546standard_V3_")

    date = filename.split('/')[1][:6]

    single_df_sample.insert(0, 'run_date', date)
    single_df_std.insert(0, 'run_date', date)

    single_df_sample["Isodat output filename"] = filename.split('/')[-1]
    single_df_std["Isodat output filename"] = filename.split('/')[-1]
    
    # The following pulls the run date from the file name. If you used a different naming convention, 
    # forgot to change the date that day, etc., fix this manually.

    df_sample = pd.concat([single_df_sample, df_sample])
    df_std = pd.concat([single_df_std, df_std])

WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero
WARNING *** OLE2 inconsistency: SSCS size is 0 but SSAT size is non-zero


## Bringing data in line
This gets rid of data from CO$_2$ peaks and lines up the Area 44 and Area 30 rows for each bottle. Check the output to make sure it's working properly.

In [15]:
def bring_in_line(df_input):

    # Adjust this to also double check peak areas?

    grouped = df_input.groupby("Time Code")

    df_new = pd.DataFrame()
    
    for group_name, df_group in grouped:
        
        if len(df_group) == 2:
            df1 = df_group.iloc[::2].reset_index()
            df2 = df_group.iloc[1::2].reset_index()
            df = df1.assign(**{"Area 30": df2["Area 30"], "rR 31NO/30NO": df2["rR 31NO/30NO"]})
    
        elif len(df_group) == 3:
            df = df_group[df_group.index % 3 != 0]
            df1 = df.iloc[::2].reset_index()
            df2 = df.iloc[1::2].reset_index()
            df = df1.assign(**{"Area 30": df2["Area 30"], "rR 31NO/30NO": df2["rR 31NO/30NO"]})
    
        df_new = pd.concat([df_new, df])
    
    df_new["Unique ID"] = df_new["Identifier 1"] + '_' + df_new["run_date"]
    df_new = df_new.set_index("Unique ID", drop=False)
    
    if not df_new.index.is_unique:
        print(df_new.index)
        raise IndexError(
            """
            Sample indices are not unique - check that the dates in your file names are unique
            and that standards from the same day have different names (e.g. 1000ppm_C).
            """)
        
    df_new = df_new.drop(columns=["index"])

    return df_new
        

In [16]:
df_sample

,run_date,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,BGD 44,Rt,FileHeader: Filename,Time Code,rR 45N2O/44N2O,rR 46N2O/44N2O,rR 31NO/30NO,Isodat output filename
0,250225,12,S2_2,0,514.732,1025.337,409.874,0.269,NaN,1.5,1806.8,Acquisition01-0020.dxf,2025/02/25 23:55:09,1.174165,1.424567,NaN,250225_V_Benguela24_St22.xls
1,250225,12,S2_2,0,-3.102,37.028,-1.896,3.506,NaN,1.9,1820.6,Acquisition01-0020.dxf,2025/02/25 23:55:09,0.775647,0.728463,NaN,250225_V_Benguela24_St22.xls
2,250225,12,S2_2,0,NaN,NaN,NaN,NaN,3.753,NaN,1820.6,Acquisition01-0020.dxf,2025/02/25 23:55:09,NaN,NaN,0.429180,250225_V_Benguela24_St22.xls
3,250225,11,AESW_0214_2,0,513.695,1025.504,409.934,0.271,NaN,1.4,1806.6,Acquisition01-0019.dxf,2025/02/25 23:22:00,1.173462,1.424669,NaN,250225_V_Benguela24_St22.xls
4,250225,11,AESW_0214_2,0,6.621,49.148,4.106,2.316,NaN,1.9,1820.6,Acquisition01-0019.dxf,2025/02/25 23:22:00,0.783099,0.737018,NaN,250225_V_Benguela24_St22.xls
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
121,250218,2,1000ppm_1,0,0.375,42.376,0.756,18.909,NaN,2.0,1801.6,Acquisition01-0001.dxf,2025/02/18 13:39:31,0.775217,0.731836,NaN,250218_V_Benguela24_Stn21.xlsx
122,250218,2,1000ppm_1,0,NaN,NaN,NaN,NaN,20.071,NaN,1801.6,Acquisition01-0001.dxf,2025/02/18 13:39:31,NaN,NaN,0.427997,250218_V_Benguela24_Stn21.xlsx
123,250218,1,1000ppm_rerun,0,514.969,1029.958,411.533,0.320,NaN,1.6,1787.6,Acquisition01-0000.dxf,2025/02/18 13:06:21,1.174669,1.428237,NaN,250218_V_Benguela24_Stn21.xlsx
124,250218,1,1000ppm_rerun,0,-4.572,41.058,0.103,1.344,NaN,2.0,1802.0,Acquisition01-0000.dxf,2025/02/18 13:06:21,0.774819,0.731486,NaN,250218_V_Benguela24_Stn21.xlsx


In [17]:
df_sample = bring_in_line(df_sample)
df_std = bring_in_line(df_std)

In [18]:
df_sample

,run_date,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,BGD 44,Rt,FileHeader: Filename,Time Code,rR 45N2O/44N2O,rR 46N2O/44N2O,rR 31NO/30NO,Isodat output filename,Unique ID
Unique ID,,,,,,,,,,,,,,,,,,
1000ppm_rerun_250218,250218,1,1000ppm_rerun,0,-4.572,41.058,0.103,1.344,1.441,2.0,1802.0,Acquisition01-0000.dxf,2025/02/18 13:06:21,0.774819,0.731486,0.426977,250218_V_Benguela24_Stn21.xlsx,1000ppm_rerun_250218
1000ppm_1_250218,250218,2,1000ppm_1,0,0.375,42.376,0.756,18.909,20.071,2.0,1801.6,Acquisition01-0001.dxf,2025/02/18 13:39:31,0.775217,0.731836,0.427997,250218_V_Benguela24_Stn21.xlsx,1000ppm_1_250218
AESW_1017_1_250218,250218,3,AESW_1017_1,0,5.115,48.801,3.934,2.470,2.639,2.0,1801.6,Acquisition01-0002.dxf,2025/02/18 14:12:40,0.779041,0.736375,0.433401,250218_V_Benguela24_Stn21.xlsx,AESW_1017_1_250218
S2_1_250218,250218,4,S2_1,0,-4.209,36.638,-2.090,3.533,3.774,1.9,1802.0,Acquisition01-0003.dxf,2025/02/18 14:45:49,0.772061,0.727803,0.429713,250218_V_Benguela24_Stn21.xlsx,S2_1_250218
Benguela24_0194_250218,250218,5,Benguela24_0194,0,8.063,51.844,5.436,3.098,3.307,1.9,1803.3,Acquisition01-0004.dxf,2025/02/18 15:18:59,0.781601,0.738549,0.434722,250218_V_Benguela24_Stn21.xlsx,Benguela24_0194_250218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0318_250225,250225,8,Benguela24_0318,0,6.525,48.387,3.730,1.756,1.884,1.9,1820.2,Acquisition01-0016.dxf,2025/02/25 21:42:31,0.783498,0.736667,0.433666,250225_V_Benguela24_St22.xls,Benguela24_0318_250225
Benguela24_0320_250225,250225,9,Benguela24_0320,0,6.914,48.805,3.936,1.788,1.917,1.9,1820.6,Acquisition01-0017.dxf,2025/02/25 22:15:41,0.783635,0.736884,0.433854,250225_V_Benguela24_St22.xls,Benguela24_0320_250225
1000ppm_2_250225,250225,10,1000ppm_2,0,0.889,42.772,0.952,18.979,20.176,2.0,1819.6,Acquisition01-0018.dxf,2025/02/25 22:48:50,0.778862,0.732568,0.428465,250225_V_Benguela24_St22.xls,1000ppm_2_250225


In [19]:
df_std

,run_date,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,BGD 44,Rt,FileHeader: Filename,Time Code,rR 45N2O/44N2O,rR 46N2O/44N2O,rR 31NO/30NO,Isodat output filename,Unique ID
Unique ID,,,,,,,,,,,,,,,,,,
1000ppm_rerun_250218,250218,1,1000ppm_rerun,0,0.348,40.866,0.008,227.112,239.221,1.8,1232.3,Acquisition01-0000.dxf,2025/02/18 13:06:21,0.778441,0.731403,0.429609,250218_V_Benguela24_Stn21.xlsx,1000ppm_rerun_250218
1000ppm_1_250218,250218,2,1000ppm_1,0,0.360,40.849,0.000,228.550,240.307,1.8,1233.3,Acquisition01-0001.dxf,2025/02/18 13:39:31,0.775175,0.730772,0.428687,250218_V_Benguela24_Stn21.xlsx,1000ppm_1_250218
AESW_1017_1_250218,250218,3,AESW_1017_1,0,0.320,40.828,-0.011,230.142,241.995,1.8,1239.2,Acquisition01-0002.dxf,2025/02/18 14:12:40,0.775360,0.730767,0.428721,250218_V_Benguela24_Stn21.xlsx,AESW_1017_1_250218
S2_1_250218,250218,4,S2_1,0,0.365,40.844,-0.003,231.710,243.622,1.8,1232.1,Acquisition01-0003.dxf,2025/02/18 14:45:49,0.775506,0.730783,0.428812,250218_V_Benguela24_Stn21.xlsx,S2_1_250218
Benguela24_0194_250218,250218,5,Benguela24_0194,0,0.340,40.843,-0.003,233.329,245.325,1.8,1235.2,Acquisition01-0004.dxf,2025/02/18 15:18:59,0.775706,0.730800,0.428865,250218_V_Benguela24_Stn21.xlsx,Benguela24_0194_250218
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0318_250225,250225,8,Benguela24_0318,0,0.287,40.827,-0.011,245.919,258.182,1.9,1230.0,Acquisition01-0016.dxf,2025/02/25 21:42:31,0.778745,0.731327,0.429912,250225_V_Benguela24_St22.xls,Benguela24_0318_250225
Benguela24_0320_250225,250225,9,Benguela24_0320,0,0.259,40.854,0.002,246.113,258.454,1.9,1227.2,Acquisition01-0017.dxf,2025/02/25 22:15:41,0.778566,0.731267,0.429836,250225_V_Benguela24_St22.xls,Benguela24_0320_250225
1000ppm_2_250225,250225,10,1000ppm_2,0,0.250,40.842,-0.004,244.891,257.173,1.9,1239.8,Acquisition01-0018.dxf,2025/02/25 22:48:50,0.778352,0.731215,0.429714,250225_V_Benguela24_St22.xls,1000ppm_2_250225


## Setting up analysis dataframe

In [ ]:
check_values = []

if len(df_sample.index) < len(df_std.index):
    for i in df_std.index:
        if i not in df_sample.index:
            check_values.append(i)
    raise IndexError(
        """
        Lengths of df_sample and df_std do not match. 
        This probably means some peaks in the sample tab didn't integrate, due to
        a needle clog or other issue.
        
        The following values are in df_std but not in df_sample: 
        """ + str(check_values))

if len(df_sample.index) > len(df_std.index):
    for i in df_sample.index:
        if i not in df_std.index:
            check_values.append(i)
    raise IndexError(
        """
        Lengths of df_sample and df_std do not match. 
        The following values are in df_sample but not in df_std: 
        """ + str(check_values))

In [20]:
df_analysis = df_sample.copy()
df_analysis = df_analysis.rename(columns={"rR 45N2O/44N2O": "rR 45N2O/44N2O sam",
                           "rR 46N2O/44N2O": "rR 46N2O/44N2O sam",
                           "rR 31NO/30NO": "rR 31NO/30NO sam"})

# Getting values from standard sheet
df_analysis["rR 45N2O/44N2O std"] = df_std["rR 45N2O/44N2O"].values
df_analysis["rR 46N2O/44N2O std"] = df_std["rR 46N2O/44N2O"].values
df_analysis["rR 31NO/30NO std"] = df_std["rR 31NO/30NO"].values

In [21]:
# Adding standard tags

df_analysis.insert(1, 'ref_tag', "")
df_analysis.loc[df_analysis.index.str.contains("S2_"), "ref_tag"] = "S2"
df_analysis.loc[df_analysis.index.str.contains("AESW"), "ref_tag"] = "AESW"
df_analysis.loc[df_analysis.index.str.contains("1000ppm"), "ref_tag"] = "1000ppm"

# Making sure the 1000 ppm reruns don't get counted
df_analysis.loc[df_analysis.index.str.contains("1000ppm_rerun"), "ref_tag"] = ""

# N$_2$O concentration calculation

In [22]:
# As written below this assumes a constant volume per sample

df_analysis['Volume'] = 0.113 #L, from Meléa's analysis
df_analysis['Volume_err'] = 0.0029 #L, from Meléa's analysis
df_analysis['N2O_nmol'] = conc_slope*df_analysis['Area 44'] + conc_int
df_analysis['N2O_nmol_err'] = conc_slope_err*df_analysis['Area 44'] + conc_int_err #CHECK THIS

df_analysis['N2O_nM'] = df_analysis['N2O_nmol']/df_analysis['Volume']
df_analysis['N2O_nM_err'] = (df_analysis['N2O_nmol_err']/df_analysis['N2O_nmol'] + 
                             df_analysis['Volume_err']/df_analysis['Volume'])*df_analysis['N2O_nM']
df_analysis


,run_date,ref_tag,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,...,Unique ID,rR 45N2O/44N2O std,rR 46N2O/44N2O std,rR 31NO/30NO std,Volume,Volume_err,N2O_nmol,N2O_nmol_err,N2O_nM,N2O_nM_err
Unique ID,,,,,,,,,,,,,,,,,,,,,
1000ppm_rerun_250218,250218,,1,1000ppm_rerun,0,-4.572,41.058,0.103,1.344,1.441,...,1000ppm_rerun_250218,0.778441,0.731403,0.429609,0.113,0.0029,1.351058,0.256801,11.956266,2.579415
1000ppm_1_250218,250218,1000ppm,2,1000ppm_1,0,0.375,42.376,0.756,18.909,20.071,...,1000ppm_1_250218,0.775175,0.730772,0.428687,0.113,0.0029,8.613466,0.561980,76.225359,6.929504
AESW_1017_1_250218,250218,AESW,3,AESW_1017_1,0,5.115,48.801,3.934,2.470,2.639,...,AESW_1017_1_250218,0.775360,0.730767,0.428721,0.113,0.0029,1.816613,0.276364,16.076221,2.858276
S2_1_250218,250218,S2,4,S2_1,0,-4.209,36.638,-2.090,3.533,3.774,...,S2_1_250218,0.775506,0.730783,0.428812,0.113,0.0029,2.256120,0.294833,19.965662,3.121535
Benguela24_0194_250218,250218,,5,Benguela24_0194,0,8.063,51.844,5.436,3.098,3.307,...,Benguela24_0194_250218,0.775706,0.730800,0.428865,0.113,0.0029,2.076265,0.287275,18.374028,3.013804
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0318_250225,250225,,8,Benguela24_0318,0,6.525,48.387,3.730,1.756,1.884,...,Benguela24_0318_250225,0.778745,0.731327,0.429912,0.113,0.0029,1.521403,0.263959,13.463745,2.681449
Benguela24_0320_250225,250225,,9,Benguela24_0320,0,6.914,48.805,3.936,1.788,1.917,...,Benguela24_0320_250225,0.778566,0.731267,0.429836,0.113,0.0029,1.534634,0.264515,13.580831,2.689374
1000ppm_2_250225,250225,1000ppm,10,1000ppm_2,0,0.889,42.772,0.952,18.979,20.176,...,1000ppm_2_250225,0.778352,0.731215,0.429714,0.113,0.0029,8.642408,0.563197,76.481484,6.946840


# Size correction

In [23]:
# Raw sample/standard calculations

df_analysis["raw 45rR/45rR"] = df_analysis["rR 45N2O/44N2O sam"]/df_analysis["rR 45N2O/44N2O std"]
df_analysis["raw 46rR/46rR"] = df_analysis["rR 46N2O/44N2O sam"]/df_analysis["rR 46N2O/44N2O std"]
df_analysis["raw 31rR/31rR"] = df_analysis["rR 31NO/30NO sam"]/df_analysis["rR 31NO/30NO std"]

# Size correction - corrected to m/z=44 peak area of 20 Vs

df_analysis["size corrected 45rR/45rR"] = (sizecorr_slope45R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 45rR/45rR"])
df_analysis["size corrected 46rR/46rR"] = (sizecorr_slope46R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 46rR/46rR"])
df_analysis["size corrected 31rR/31rR"] = (sizecorr_slope31R * (20 - df_analysis["Area 44"]) 
                                           + df_analysis["raw 31rR/31rR"])

In [24]:
df_analysis

,run_date,ref_tag,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,...,N2O_nmol,N2O_nmol_err,N2O_nM,N2O_nM_err,raw 45rR/45rR,raw 46rR/46rR,raw 31rR/31rR,size corrected 45rR/45rR,size corrected 46rR/46rR,size corrected 31rR/31rR
Unique ID,,,,,,,,,,,,,,,,,,,,,
1000ppm_rerun_250218,250218,,1,1000ppm_rerun,0,-4.572,41.058,0.103,1.344,1.441,...,1.351058,0.256801,11.956266,2.579415,0.995347,1.000113,0.993873,0.996051,1.000865,0.994241
1000ppm_1_250218,250218,1000ppm,2,1000ppm_1,0,0.375,42.376,0.756,18.909,20.071,...,8.613466,0.561980,76.225359,6.929504,1.000054,1.001456,0.998391,1.000095,1.001500,0.998412
AESW_1017_1_250218,250218,AESW,3,AESW_1017_1,0,5.115,48.801,3.934,2.470,2.639,...,1.816613,0.276364,16.076221,2.858276,1.004748,1.007674,1.010918,1.005409,1.008381,1.011263
S2_1_250218,250218,S2,4,S2_1,0,-4.209,36.638,-2.090,3.533,3.774,...,2.256120,0.294833,19.965662,3.121535,0.995559,0.995922,1.002101,0.996180,0.996586,1.002426
Benguela24_0194_250218,250218,,5,Benguela24_0194,0,8.063,51.844,5.436,3.098,3.307,...,2.076265,0.287275,18.374028,3.013804,1.007600,1.010605,1.013656,1.008238,1.011286,1.013990
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0318_250225,250225,,8,Benguela24_0318,0,6.525,48.387,3.730,1.756,1.884,...,1.521403,0.263959,13.463745,2.681449,1.006104,1.007301,1.008732,1.006793,1.008037,1.009092
Benguela24_0320_250225,250225,,9,Benguela24_0320,0,6.914,48.805,3.936,1.788,1.917,...,1.534634,0.264515,13.580831,2.689374,1.006510,1.007680,1.009348,1.007197,1.008414,1.009707
1000ppm_2_250225,250225,1000ppm,10,1000ppm_2,0,0.889,42.772,0.952,18.979,20.176,...,8.642408,0.563197,76.481484,6.946840,1.000655,1.001850,0.997093,1.000694,1.001892,0.997114


# Scale normalization

In [25]:
std_const

,ref_tag,d15Na,d15Nb,d18O,d15Nbulk,SP,15Ralpha,15Rbeta,18R,17R,45R,46R,45R/45R,46R/46R
S2,S2,5.55,-12.87,32.73,-3.66,18.42,0.003697,0.003629,0.002071,0.000386,0.007712,0.002087,0.996296,0.993233
AESW,AESW,15.6,-2.3,44.4,6.65,17.9,0.003734,0.003668,0.002094,0.000389,0.00779,0.002111,1.00638,1.004525
1000ppm,1000ppm,-0.11,-0.45,37.83,-0.28,0.34,0.003676,0.003675,0.002081,0.000387,0.007738,0.002097,0.999634,0.998152


In [26]:
# Selecting standards

scale_norm = pd.DataFrame()

scale_norm["ref_tag"] = df_analysis["ref_tag"]

scale_norm["Measured 45R/45R"] = df_analysis['size corrected 45rR/45rR'].where(
    scale_norm["ref_tag"].isin(["S2", "AESW", "1000ppm"]))

scale_norm["Measured 46R/46R"] = df_analysis['size corrected 46rR/46rR'].where(
    scale_norm["ref_tag"].isin(["S2", "AESW", "1000ppm"]))

scale_norm = scale_norm.dropna()

In [27]:
# Populating with comparison values

scale_norm.loc[scale_norm["ref_tag"] == "S2", "Known 45R/45R"] = std_const.at["S2", "45R/45R"]
scale_norm.loc[scale_norm["ref_tag"] == "AESW", "Known 45R/45R"] = std_const.at["AESW", "45R/45R"]
scale_norm.loc[scale_norm["ref_tag"] == "1000ppm", "Known 45R/45R"] = std_const.at["1000ppm", "45R/45R"]

scale_norm.loc[scale_norm["ref_tag"] == "S2", "Known 46R/46R"] = std_const.at["S2", "46R/46R"]
scale_norm.loc[scale_norm["ref_tag"] == "AESW", "Known 46R/46R"] = std_const.at["AESW", "46R/46R"]
scale_norm.loc[scale_norm["ref_tag"] == "1000ppm", "Known 46R/46R"] = std_const.at["1000ppm", "46R/46R"]

scale_norm["ln(measured 45R/45R)"] = np.log(scale_norm["Measured 45R/45R"])
scale_norm["ln(measured 46R/46R)"] = np.log(scale_norm["Measured 46R/46R"])
scale_norm["ln(known 45R/45R)"] = np.log(scale_norm["Known 45R/45R"])
scale_norm["ln(known 46R/46R)"] = np.log(scale_norm["Known 46R/46R"])

In [28]:
scale_norm

,ref_tag,Measured 45R/45R,Measured 46R/46R,Known 45R/45R,Known 46R/46R,ln(measured 45R/45R),ln(measured 46R/46R),ln(known 45R/45R),ln(known 46R/46R)
Unique ID,,,,,,,,,
1000ppm_1_250218,1000ppm,1.000095,1.001500,0.999634,0.998152,0.000095,0.001499,-0.000366,-0.001850
AESW_1017_1_250218,AESW,1.005409,1.008381,1.006380,1.004525,0.005395,0.008346,0.006359,0.004514
S2_1_250218,S2,0.996180,0.996586,0.996296,0.993233,-0.003827,-0.003419,-0.003711,-0.006790
1000ppm_2_250218,1000ppm,1.000901,1.001889,0.999634,0.998152,0.000901,0.001887,-0.000366,-0.001850
AESW_0117_2_250218,AESW,1.006884,1.008740,1.006380,1.004525,0.006860,0.008702,0.006359,0.004514
S2_2_250218,S2,0.997519,0.997320,0.996296,0.993233,-0.002484,-0.002684,-0.003711,-0.006790
1000ppm_3_250218,1000ppm,1.000979,1.001688,0.999634,0.998152,0.000978,0.001686,-0.000366,-0.001850
AESW_0117_3_250218,AESW,1.006196,1.008810,1.006380,1.004525,0.006177,0.008772,0.006359,0.004514
S2_3_250218,S2,0.996400,0.997148,0.996296,0.993233,-0.003607,-0.002856,-0.003711,-0.006790


## <span style="color:cyan">INPUT: Check plots and remove outlier values</span>
### List of standards to drop throughout

In [29]:
drop_standards = []
#drop_standards=["1000ppm_2_241106"]

scale_norm = scale_norm.drop(drop_standards)

### 45R plot

In [29]:
fit45 = np.polyfit(scale_norm["ln(measured 45R/45R)"], scale_norm["ln(known 45R/45R)"], 1, full=True)
m45 = fit45[0][0]
b45 = fit45[0][1]
err45 = fit45[1][0] # error to plot here?

print("45R slope: " + str(m45) + "\n45R intercept: " + str(b45))

45R slope: 1.0551317001432743
45R intercept: -0.0002765870913087642


In [30]:
p = bokeh.plotting.figure(
    max_width=500,
    height=300,
    title="45R/45R Scale Normalization",
    x_axis_label="ln(measured 45R/45R)",
    y_axis_label="ln(known 45R/45R)"
)

hvr = bokeh.models.HoverTool(tooltips="@{Unique ID}")
p.add_tools(hvr)

scatter = p.scatter("ln(measured 45R/45R)", "ln(known 45R/45R)", size=10, source=scale_norm)
hvr.renderers = [scatter]

x_vals = np.linspace(scale_norm["ln(measured 45R/45R)"].min(), scale_norm["ln(measured 45R/45R)"].max())
bestfit = p.line(x_vals, np.poly1d([m45,b45])(x_vals), legend_label=f"y = {m45:.4f}x + {b45:.4f}")

p.legend.location = (20, 160)

bokeh.plotting.show(p)

If any standards are bad, add their names to `drop_standards` a few cells up and rerun this whole section.

### 46R plot

In [32]:
fit46 = np.polyfit(scale_norm["ln(measured 46R/46R)"], scale_norm["ln(known 46R/46R)"], 1, full=True)
m46 = fit46[0][0]
b46 = fit46[0][1]
err46 = fit46[1][0]

print("46R slope: " + str(m46) + "\n46R intercept: " + str(b46))

46R slope: 1.0053904076881122
46R intercept: -0.003663140776523598


In [33]:
p = bokeh.plotting.figure(
    max_width=500,
    height=300,
    title="46R/46R Scale Normalization",
    x_axis_label="ln(measured 46R/46R)",
    y_axis_label="ln(known 46R/46R)",
)

hvr = bokeh.models.HoverTool(tooltips="@{Unique ID}")
p.add_tools(hvr)

scatter = p.scatter("ln(measured 46R/46R)", "ln(known 46R/46R)", size=10, source=scale_norm)
hvr.renderers = [scatter]

x_vals = np.linspace(scale_norm["ln(measured 46R/46R)"].min(), scale_norm["ln(measured 46R/46R)"].max())
bestfit = p.line(x_vals, np.poly1d([m46,b46])(x_vals), legend_label=f"y = {m46:.4f}x + {b46:.4f}")
p.legend.location = (20, 160)

bokeh.plotting.show(p)

If any standards are bad, add their names to `drop_standards` a few cells up and rerun this whole section.

## Apply scale normalization 

In [35]:
df_analysis["scale decompressed 45rR/45rR"] = df_analysis["size corrected 45rR/45rR"]**m45 * np.exp(b45)
df_analysis["scale decompressed 46rR/46rR"] = df_analysis["size corrected 46rR/46rR"]**m46 * np.exp(b46)

## Multiply by R values from reference N2O tank

In [36]:
df_analysis["size corrected 31R"] = df_analysis["size corrected 31rR/31rR"]*N2O_ref31R
df_analysis["size corrected 45R"] = df_analysis["scale decompressed 45rR/45rR"]*N2O_ref45R
df_analysis["size corrected 46R"] = df_analysis["scale decompressed 46rR/46rR"]*N2O_ref46R

# Interface with pyisotopomer

In [37]:
df_analysis["D17O"] = 0. # Replace with known values if you have them, but 0 is fine

In [52]:
# This is from the earlier scale normalization step - make sure it's accurate

print(drop_standards)

[]


In [39]:
df_analysis = df_analysis.drop(drop_standards)
df_analysis

,run_date,ref_tag,Row,Identifier 1,Is Ref _,d 15N/14N,d 18O/16O,d 17O/16O,Area 44,Area 30,...,raw 31rR/31rR,size corrected 45rR/45rR,size corrected 46rR/46rR,size corrected 31rR/31rR,scale decompressed 45rR/45rR,scale decompressed 46rR/46rR,size corrected 31R,size corrected 45R,size corrected 46R,D17O
Unique ID,,,,,,,,,,,,,,,,,,,,,
1000ppm_rerun_250218,250218,,1,1000ppm_rerun,0,-4.572,41.058,0.103,1.344,1.441,...,0.993873,0.996051,1.000865,0.994241,0.995558,0.997210,0.003712,0.007707,0.002095,0.0
1000ppm_1_250218,250218,1000ppm,2,1000ppm_1,0,0.375,42.376,0.756,18.909,20.071,...,0.998391,1.000095,1.001500,0.998412,0.999824,0.997846,0.003728,0.007740,0.002097,0.0
AESW_1017_1_250218,250218,AESW,3,AESW_1017_1,0,5.115,48.801,3.934,2.470,2.639,...,1.010918,1.005409,1.008381,1.011263,1.005430,1.004739,0.003776,0.007783,0.002111,0.0
S2_1_250218,250218,S2,4,S2_1,0,-4.209,36.638,-2.090,3.533,3.774,...,1.002101,0.996180,0.996586,1.002426,0.995694,0.992924,0.003743,0.007708,0.002086,0.0
Benguela24_0194_250218,250218,,5,Benguela24_0194,0,8.063,51.844,5.436,3.098,3.307,...,1.013656,1.008238,1.011286,1.013990,1.008415,1.007649,0.003786,0.007806,0.002117,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0318_250225,250225,,8,Benguela24_0318,0,6.525,48.387,3.730,1.756,1.884,...,1.008732,1.006793,1.008037,1.009092,1.006890,1.004394,0.003768,0.007794,0.002111,0.0
Benguela24_0320_250225,250225,,9,Benguela24_0320,0,6.914,48.805,3.936,1.788,1.917,...,1.009348,1.007197,1.008414,1.009707,1.007317,1.004772,0.003770,0.007798,0.002111,0.0
1000ppm_2_250225,250225,1000ppm,10,1000ppm_2,0,0.889,42.772,0.952,18.979,20.176,...,0.997093,1.000694,1.001892,0.997114,1.000455,0.998238,0.003723,0.007745,0.002098,0.0


## Run scrambling

In [40]:
# Add the kwarg outputfile="filename.csv" if you want to control save location,
# or saveout=False if you don't want to save a file.

scrambling_output = pyisotopomer.Scrambling(inputdf=df_analysis, refdf=std_const, method="least_squares",
                                            initialguess=[gamma_initial_guess, kappa_initial_guess],
                                           outputfile=scrambling_output_file).alloutputs

output saved as 250531_scrambling_output.xlsx
no matches found for ['']
scrambling calculated with least squares solver


/Users/levr/code/pyisotopomer-dev/src/pyisotopomer/parseoutput.py:157: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  maindf = pd.concat([maindf, df])


In [41]:
scrambling_output

,ref_tag_1,size corrected 31R_1,size corrected 45R_1,size corrected 46R_1,15Rbulk_1,17R_1,ref_tag_2,size corrected 31R_2,size corrected 45R_2,size corrected 46R_2,15Rbulk_2,17R_2,gamma,kappa,error31r_ref1_permil,error31r_ref2_permil
250218,AESW,0.003776,0.007783,0.002111,0.003697,0.000389,S2,0.003743,0.007708,0.002086,0.003661,0.000386,0.179583,0.088705,0.404270,-0.418337
250218,AESW,0.003776,0.007783,0.002111,0.003697,0.000389,S2,0.003742,0.007719,0.002088,0.003666,0.000386,0.179609,0.088594,0.270688,-0.281326
250218,AESW,0.003776,0.007783,0.002111,0.003697,0.000389,S2,0.003737,0.007709,0.002088,0.003662,0.000386,0.179753,0.088048,-0.400650,0.407266
250218,AESW,0.003775,0.007795,0.002112,0.003703,0.000389,S2,0.003743,0.007708,0.002086,0.003661,0.000386,0.179591,0.088654,0.460623,-0.475363
250218,AESW,0.003775,0.007795,0.002112,0.003703,0.000389,S2,0.003742,0.007719,0.002088,0.003666,0.000386,0.179617,0.088542,0.327013,-0.338379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250224,1000ppm,0.003724,0.007747,0.002098,0.003680,0.000387,AESW,0.003766,0.007795,0.002111,0.003703,0.000389,0.179900,0.087173,-0.380332,1.211315
250225,1000ppm,0.003721,0.007736,0.002096,0.003674,0.000387,AESW,0.003765,0.007780,0.002109,0.003696,0.000388,0.181194,0.087513,-0.599665,0.586502
250225,1000ppm,0.003721,0.007736,0.002096,0.003674,0.000387,AESW,0.003768,0.007795,0.002112,0.003703,0.000389,0.180575,0.087340,-0.159888,0.156264
250225,1000ppm,0.003723,0.007745,0.002098,0.003679,0.000387,AESW,0.003765,0.007780,0.002109,0.003696,0.000388,0.181347,0.087905,-0.836078,0.816512


In [42]:
gamma_mean = scrambling_output['gamma'].mean()
kappa_mean = scrambling_output['kappa'].mean()

print("gamma: " + str(gamma_mean) + "\nkappa: " + str(kappa_mean))

gamma: 0.18010280529575212
kappa: 0.0878418458670568


In [43]:
df_analysis['gamma'] = gamma_mean
df_analysis['kappa'] = kappa_mean

## Run isotopomers

In [44]:
# Add the kwarg outputfile="filename.csv" if you want to control save location,
# or saveout=False if you don't want to save a file.

isotopomersoutput = pyisotopomer.Isotopomers(inputdf=df_analysis, outputfile=isotopomers_output_file).deltavals

output saved as 250531_isotopeoutput.csv


In [45]:
isotopomersoutput

,run_date,Identifier 1,d15Na,d15Nb,SP,d15Nbulk,d17O,d18O
0,250218,1000ppm_rerun,-3.451187,-5.641593,2.190406,-4.546390,18.877093,36.907268
1,250218,1000ppm_1,1.218268,-1.361390,2.579658,-0.071561,19.180971,37.506685
2,250218,AESW_1017_1,17.170270,-5.881290,23.051560,5.644490,22.790915,44.640325
3,250218,S2_1,8.170892,-16.741830,24.912723,-4.285469,16.597424,32.415843
4,250218,Benguela24_0194,20.004497,-2.587929,22.592425,8.708284,24.306222,47.641784
...,...,...,...,...,...,...,...,...
158,250225,Benguela24_0318,13.812035,0.570037,13.241998,7.191036,22.596408,44.255355
159,250225,Benguela24_0320,14.532308,0.728516,13.803792,7.630412,22.793153,44.644756
160,250225,1000ppm_2,-0.770868,1.936426,-2.707293,0.582779,19.384121,37.907504
161,250225,AESW_0214_2,13.896767,0.602696,13.294071,7.249732,22.955090,44.965314


# Make a nice final file with just your data
Includes original file name, delta and site preference values, and N2O concentrations.

In [46]:
df_output = isotopomersoutput.copy()
df_output["Unique ID"] = df_analysis["Unique ID"].values
df_output= df_output.set_index("Unique ID")

In [47]:
# Grabbing any extra columns you want from df_analysis - add others if desired

cols_to_add = ['Isodat output filename', 'N2O_nmol', 'N2O_nmol_err', 'N2O_nM', 'N2O_nM_err']

df_output = pd.merge(df_output, df_analysis[cols_to_add], on="Unique ID")

# Rearranging order

df_output.insert(0, 'Isodat output filename', df_output.pop('Isodat output filename'))

In [48]:
# Dropping rows with standards

df_output = df_output[~df_output.index.str.contains("S2_|AESW|1000ppm")]


In [49]:
df_output

,Isodat output filename,run_date,Identifier 1,d15Na,d15Nb,SP,d15Nbulk,d17O,d18O,N2O_nmol,N2O_nmol_err,N2O_nM,N2O_nM_err
Unique ID,,,,,,,,,,,,,
Benguela24_0194_250218,250218_V_Benguela24_Stn21.xlsx,250218,Benguela24_0194,20.004497,-2.587929,22.592425,8.708284,24.306222,47.641784,2.076265,0.287275,18.374028,3.013804
Benguela24_0196_250218,250218_V_Benguela24_Stn21.xlsx,250218,Benguela24_0196,19.450431,-2.692308,22.142739,8.379061,24.157554,47.347125,2.065515,0.286823,18.278896,3.007365
Benguela24_0198_250218,250218_V_Benguela24_Stn21.xlsx,250218,Benguela24_0198,18.412460,-1.929733,20.342193,8.241364,23.693930,46.428478,2.183351,0.291775,19.321691,3.077947
Benguela24_0200_250218,250218_V_Benguela24_Stn21.xlsx,250218,Benguela24_0200,18.652159,-2.134333,20.786491,8.258913,23.717337,46.474849,2.406205,0.301140,21.293854,3.211434
Benguela24_0202_250218,250218_V_Benguela24_Stn21.xlsx,250218,Benguela24_0202,19.128108,-1.514618,20.642726,8.806745,23.764681,46.568643,2.999933,0.326089,26.548076,3.567069
...,...,...,...,...,...,...,...,...,...,...,...,...,...
Benguela24_0312_250225,250225_V_Benguela24_St22.xls,250225,Benguela24_0312,12.610738,-2.736228,15.346966,4.937255,21.786203,42.652528,1.766171,0.274244,15.629831,2.828062
Benguela24_0314_250225,250225_V_Benguela24_St22.xls,250225,Benguela24_0314,11.246283,-2.363673,13.609956,4.441305,22.548099,44.159751,1.666527,0.270057,14.748029,2.768377
Benguela24_0316_250225,250225_V_Benguela24_St22.xls,250225,Benguela24_0316,13.967416,-2.317178,16.284594,5.825119,22.498620,44.061839,1.520576,0.263924,13.456427,2.680954


In [50]:
# Writing to file

df_output.to_csv(processed_output_file)

#df_output.to_excel("test.xlsx")